In [104]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
# from badx12 import Parser

from datetime import datetime
import pandas as pd

os.uname()

posix.uname_result(sysname='Linux', nodename='Acer-JC', release='5.15.133.1-microsoft-standard-WSL2', version='#1 SMP Thu Oct 5 21:02:42 UTC 2023', machine='x86_64')

Now spin up the Selenium-GoogleChrome Docker container. 

`docker start <container id or name>`

In [ ]:
# Get the path of chromedriver which you have install

def parse_creation_date(datetime_str: str) -> datetime:
    # 6/28/24 11:34 AM => 6/28/2024 11:34 AM
    date_part, time_part, meridiem = datetime_str.split(' ')
    month, day, year = date_part.split('/')
    full_year = f"20{year}"
    new_datetime_str = f"{month}/{day}/{full_year} {time_part} {meridiem}"
    date_format = "%m/%d/%Y %I:%M %p"
    creation_date = datetime.strptime(new_datetime_str, date_format)
    return creation_date

def startScrapeBot_byRawX12(username, password, url, last_crawled_datetime, brieftest=False):
    print('Start scrape bot byRawX12....')
    if brieftest:
        print('It is a Brief Test! ')
    # path = "./chrome-win64/chrome.exe"
    # giving the path of chromedriver to selenium webdriver
    driver = webdriver.Remote(
        command_executor='http://localhost:4444/wd/hub',
        options=webdriver.ChromeOptions()
    )

    # set waiting page timeout for atmost 60 seconds
    wait = WebDriverWait(driver, 60)

    # opening the website  in chrome.
    print('Opening URL....')
    try:
        driver.get(url)
        assert "iExchangeWeb" in driver.title
    except:
        driver.quit()

    print('find the id "login-box"....')
    try:
        loginBox = driver.find_element(By.ID,            
            "login-box")
    except:
        driver.quit()
    
    # find the id or name or class of
    # username by inspecting on username input
    print('find the id "userName" within loginBox....')
    try:
        loginBox.find_element(By.ID,
            "userName").send_keys(username)
    except:
        driver.quit()
     
    # find the password by inspecting on password input
    print('find the id "password"....')
    try:
        loginBox.find_element(By.ID,            
            "password").send_keys(password)
    except:
        driver.quit()
    
    
    # click on submit
    print('clicking on submit after finding userName and password....')
    try:
        signin_button = loginBox.find_element(By.CSS_SELECTOR,
            "#login-box .btn-primary")
        assert signin_button.text == 'Sign In'
        signin_button.click()
    except:
        driver.quit()

    # Locate the <tbody> tag where the table is at
    try:
        # WebDriverWait(driver, 10).until(EC.url_changes(url))
        tbody = WebDriverWait(driver, 60).until(
            EC.visibility_of_element_located((By.TAG_NAME, 'tbody'))
        )
        mailbox_url = driver.current_url
        assert 'mailbox/inbox' in mailbox_url
        print('successful login to mailbox!')
    except:
        driver.quit() 


    try:        
        # Get all the rows that all contain a <tr> tag
        rows = tbody.find_elements(By.TAG_NAME, 'tr')
        print(f'rows = {len(rows)}')
    except:
        print('error at: Get all the rows that all contain a <tr> tag')
        driver.quit()

    # Out of all the rows, find the "un-crawled" rows
    new_rows=[]
    # try:
    
    for i, row in enumerate(rows):
        print(f'getting uncrawled row {i}....')        
        if brieftest and i==5: break

        try:
            creation_date_str = row.find_element(By.XPATH, 
                                                    "./td[11]").text
            print('found creation_date_str....')
        except:
            print('error at: creation_date_str = row.find_element(By.XPATH,"./td[11]").text')
            driver.quit()
        
        try:
            creation_date = parse_creation_date(creation_date_str)
            print('parsed creation date....')
        except:
            print('error at: parse_creation_date')
            driver.quit()
        if last_crawled_datetime >= creation_date:
            break
        new_rows.append(row)



    # in each un-crawled row, click on its "view" folder-like button and collect the raw X12-format EDI data. 
    for i, row in enumerate(reversed(new_rows)): # reversed, start processing from earliest non-crawled date.
        # click on View Button and direct to EDI item page. 
        try:
            view_button = WebDriverWait(driver, 60).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '[title="View"]'))
            )
            view_button.click()
            
            assert 'item' in driver.current_url
            print('successfully accessed EDI item data page! ')
        except:
            print('error at: click on its "view" folder-like button')
            driver.quit()


        # Now we about to crawl raw X12-format EDI data and pipe it to excel and also a Postgres Database

        # click on view source code button
        try:
            view_source_data_button = WebDriverWait(driver, 60).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '[title="View Source Data"]'))
            )
            view_source_data_button.click()

            # check if "Source Data" title is present
            # Need to Switch to iFrame first, because #document is under an iframe! 
            # Find the iframe element by id
            iframe_locator = driver.find_element(By.ID, "contentFrame")
            # Switch to the iframe context
            WebDriverWait(driver, 60).until(
                # If the frame is available it switches the given driver to the specified frame.
                EC.frame_to_be_available_and_switch_to_it(iframe_locator)
            )
            source_data_text = driver.find_element(By.XPATH, '/html/body/h1').text
            assert source_data_text == 'Source Data'
            print('Source Data is Here!')

        except:
            print('error at: click on view_source_data_button')
            driver.quit()


        # Go to radio buttons and select view mode "Raw"
        # PARENT: <div id=RawData_RenderedContentFrame
        # CHILD: <div id=RawData_ViewModes
        # GRANDCHILD: <span id=RawData_ViewRaw

        # PARENT: <div id=RawData_RenderedContentFrame
        try:
            rawdata_content_frame = driver.find_element(By.ID, "RawData_RenderedContentFrame")
            print('found RawData_RenderedContentFrame!!')
        except NoSuchElementException:
            print("RawData_RenderedContentFrame not found")
            driver.quit() 
        
        # CHILD: <div id=RawData_ViewModes
        try:
            rawdata_view_modes = rawdata_content_frame.find_element(By.ID, "RawData_ViewModes")
            assert rawdata_view_modes.text.startswith("View Mode:"), "rawdata_view_modes.text doesn't startwith 'View Mode:'"
            print('found RawData_ViewModes!!')
        except NoSuchElementException:
            print("RawData_ViewModes not found, or rawdata_view_modes.text doesn't startwith 'View Mode:")
            driver.quit() 
                
        # GRANDCHILD: <span id=RawData_ViewRaw
        try:
            rawdata_radio_button = WebDriverWait(driver, 60).until(
                # finds an element with the ID RawData_ViewRaw within the subtree of the current context node.
                EC.element_to_be_clickable((By.XPATH, ".//*[@id='RawData_ViewRaw']"))
            )
            rawdata_radio_button.click()
        except:
            print('error at: rawdata_radio_button = WebDriverWait(driver, 60).until(')
            driver.quit()
        

        # get raw X12-format EDI data 
        # PARENT: <div id=RawData_RenderedContentFrame
        # CHILD: <div id=RawData_RenderedContent
        # GRANDCHILD: <pre id=RawData_RenderedContentPre
        
        # iframe context changes after clicking radio button "Raw", finding RawData_RenderedContentPre 
        # should better search from the beginning of iframe . 
        try:            
            rawdata_content_pre = driver.find_element(By.XPATH, "//*[@id='RawData_RenderedContentPre']")
            rawdata_X12_EDI = rawdata_content_pre.text
            print(rawdata_X12_EDI)
        except:
            print('error at: get raw X12-format EDI data')
            driver.quit()   
        


        # Decode X12-format and Insert into excel row by row
        # Python X12 packages does not have enough documentation. 
        # the results are not that clear
        # Parse by myself without using any raw source data? find the CSS classes: caption, data
        # How to use pyx12 or badX12 ?
        

        

        # switch back to main mailbox page
        try:    
            # if needed to switchback to original parent frame from iframe
            # driver.switch_to.default_content()

            # Navigate back to the previous page
            driver.get(mailbox_url)

            # Wait until the URL changes back to the previous one (optional)
            WebDriverWait(driver, 60).until(EC.url_to_be(mailbox_url))
            assert driver.current_url == mailbox_url
            print(f'directed back to mailbox page {driver.current_url}! ')
        except:
            print('error at: back to mailbox page')
            driver.quit()
    
    driver.quit()
            




In [111]:
def startScrapeBot_byHTMLclass(username, password, url, last_crawled_datetime, brieftest=False, brieftestLoops=20):
    print('Start scrape bot byHTMLclass....')
    if brieftest:
        print('It is a Brief Test! ')
    # path = "./chrome-win64/chrome.exe"
    # giving the path of chromedriver to selenium webdriver
    driver = webdriver.Remote(
        command_executor='http://localhost:4444/wd/hub',
        options=webdriver.ChromeOptions()
    )

    # opening the website  in chrome.
    print('Opening URL....')
    try:
        driver.get(url)
        assert "iExchangeWeb" in driver.title
    except:
        driver.quit()

    print('find the id "login-box"....')
    try:
        loginBox = WebDriverWait(driver, 60).until( \
            EC.presence_of_element_located((By.ID, "login-box")))
        print('found loginbox!')
    except:
        driver.quit()
    
    # find the id or name or class of
    # username by inspecting on username input
    print('find the id "userName" within loginBox....')
    try:
        userNameForm = WebDriverWait(loginBox, 60).until( \
            EC.presence_of_element_located((By.ID, "userName")))
        print('found userNameForm!')
        userNameForm.send_keys(username)
        print('sent userName!')
    except:
        print('error at: have not sent userName')
        driver.quit()
     
    # find the password by inspecting on password input
    print('find the id "password" within loginBox....')
    try:
        passwordForm = WebDriverWait(loginBox, 60).until( \
            EC.presence_of_element_located((By.ID, "password")))
        print('found password!')
        passwordForm.send_keys(password)
        print('sent password!')
    except:
        print('error at: have not sent password')
        driver.quit()
    
    # click on submit
    print('clicking on submit after finding userName and password....')
    try:
        signin_button = WebDriverWait(loginBox, 60).until(
                            EC.element_to_be_clickable((By.CSS_SELECTOR, "#login-box .btn-primary"))
                        )
        print('found signin_button! ')
    except Exception as e:
        print(f'error at: signin_button = WebDriverWait(driver, 60).until( \n{e}')
        driver.quit()

    try:
        assert signin_button.text == 'Sign In'
        print('Assured signin button has the text "Sign in", prepare to click!')
        signin_button.click()
    except Exception as e:
        print('error at: assert signin_button.text == Sign in')
        print(e)
        driver.quit()

    # New Page: mailbox/inbox
    try:
        assert WebDriverWait(driver, 60).until(EC.url_contains('mailbox/inbox'))
        print('successful login to mailbox!')
    except Exception as e:
        print(f'error at: "assert mailbox/inbox in mailbox_url"')
        print(e)
        driver.quit()

    # Locate the sentmail_button
    try:
        leftside_bar = WebDriverWait(driver, 60).until( \
            EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/aside[1]')))
        mailbox_button = WebDriverWait(leftside_bar, 60).until( \
            EC.presence_of_element_located((By.XPATH, './/section/ul/li[1]/a')))
        # find_element(By.XPATH, '/html/body/div[2]/aside[1]/section/ul/li[1]/a')
        print('found mailbox_button, prepare to click! ')
        mailbox_button.click()
    except NoSuchElementException as nsee:
        print('error at: left_sidebar = driver.find_element...')
        print(nsee)
        driver.quit()

    try:
        sentmail_button = WebDriverWait(leftside_bar, 60).until(
                            EC.element_to_be_clickable((By.XPATH, ".//section/ul/li[1]/ul/li[4]/a"))
                        )
        print('found Sent button! ')
    except Exception as e:
        print('error at: sentmail_button = WebDriverWait(driver, 60).unti')
        print(e)
        driver.quit() 

    try:
        assert WebDriverWait(sentmail_button, 60).until(
                            EC.presence_of_element_located((By.XPATH, ".//span"))
                        ).text == "Sent"
        print('Assured that the text "Sent" is present in the button!, prepared to click...')
        sentmail_button.click()
    except Exception as e:
        print('error at: assert sentmail_button.find_element(By.XPATH, ')
        print(e)
        driver.quit() 
    
    try:
        assert WebDriverWait(driver, 60).until(EC.url_contains('mailbox/sent'))
        print('Navigated to Sent mail page!')
        assert WebDriverWait(driver, 60).until(
            EC.text_to_be_present_in_element((By.XPATH, "/html/body/div[2]/aside[2]/ol/li[2]"), "Sent")
        )
        print('Mailbox/Sent is in title! ')
        sentmail_url = driver.current_url
        print(sentmail_url)
    except Exception as e:
        print('error at: assert WebDriverWait(driver, 60).until(EC.url_contains(mailbox/sent)) or is_title_sent')
        print(e)
        driver.quit()
    
    # New page, driver current page redirected! 
    # Inside sent mails, find the rows where Subject="Accepted -Ship Notice....."
    
    # Locate the <tbody> tag where the table is at
    try:
        tbody = WebDriverWait(driver, 60).until(
            EC.visibility_of_element_located((By.TAG_NAME, 'tbody'))
        )
        print('found tbody tag! ')
    except:
        driver.quit()
    # Get all the rows that all contain a <tr> tag
    try:     
        rows = WebDriverWait(tbody, 60).until(
            EC.presence_of_all_elements_located((By.TAG_NAME, 'tr'))
        )
        print(f'got all rows! row count={len(rows)}')
    except:
        print('error at: Get all the rows that all contain a <tr> tag')
        driver.quit()

    # Out of all the rows, find the "un-crawled" rows
    ship_notice_idx=[]    
    for i, row in enumerate(rows):
        print(f'getting uncrawled row {i}....')
        if brieftest and i==brieftestLoops: break

        try:
            creation_date_str = WebDriverWait(row, 60).until( \
                EC.presence_of_element_located((By.XPATH, "./td[11]"))).text
            creation_date = parse_creation_date(creation_date_str)
            if last_crawled_datetime >= creation_date:
                print('last_crawled_datetime >= creation_date. done. ')
                break
            print(f'found and parsed creation_date_str: {creation_date}')
            subject_str = WebDriverWait(row, 60).until( \
                EC.presence_of_element_located((By.XPATH, "./td[10]"))).text
            # print(f'found subject_str: {subject_str}')
        except Exception as e:
            print('error at: subject_str = row.find_element(By.XPATH,"./td[10]").text')
            print(e)
            driver.quit()
        
        try:
            if subject_str.startswith('Accepted -Ship Notice'):
                print(f'{subject_str}')
                ship_notice_idx.append(i)
            else: continue
            
        except Exception as e:
            print('error at: subject_str.startswith("Accepted -Ship Notice")')
            print(e)
            driver.quit()
    
    # In each un-crawled ship notice, click on its "view" folder-like button and get data from classes = 'caption', 'data'. 
    json_data_list = []
    for idx in reversed(ship_notice_idx): # reversed, start processing from earliest non-crawled date.
        # try:
        #     subject_str = WebDriverWait(row, 60).until( 
        #             EC.presence_of_element_located((By.XPATH, "./td[10]"))).text
        #     print(f'subject_str = {subject_str} ./td[10]')
        # except Exception as e:
        #     print(f'error at: subject_str = {subject_str} ./td[10]')
        #     print(e)
        #     driver.quit()
        
        assert WebDriverWait(driver, 60).until(EC.visibility_of_all_elements_located((By.TAG_NAME, "tbody")))
        print('waited for tbody to appear in the mailbox/sent page')
        


        # To avoid stale element exception, find all rows everytime. 
        try:
            tbody = WebDriverWait(driver, 60).until(
                EC.visibility_of_element_located((By.TAG_NAME, 'tbody'))
            )
            rows = WebDriverWait(tbody, 60).until(
                EC.presence_of_all_elements_located((By.TAG_NAME, 'tr'))
            )
        except Exception as e:
            print('error at: avoid stale element exception, find all rows everytime. ')
            print(e)
            driver.quit()

        row = rows[idx]

        # click on View Button and direct to EDI item page. 
        try:
            view_button = WebDriverWait(row, 60).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '[title="View"]'))
            )
            view_button.click()
        except Exception as e:
            print('error at: view_button click!')
            print(e)
            driver.quit()

        try:
            assert WebDriverWait(driver, 60).until(
                EC.url_contains('mailbox/item')
            )
            print('url contains mailbox/item (inside view)')
            assert WebDriverWait(driver, 60).until( \
                EC.visibility_of_all_elements_located((By.XPATH, "/html/body/div[2]/aside[2]/section"))
            )
            print(f'visibility_of_all_elements_located((By.XPATH, "/html/body/div[2]/aside[2]/section")')
            assert WebDriverWait(driver, 60).until(
                EC.presence_of_element_located((By.ID, 'contentFrame'))
            ), "contentFrame is NOT seen!!!"
            print('contentFrame is Seen! ')

            
            
        except Exception as e:
            print('error at: assert WebDriverWait(driver, 60).until(EC.url_contains(mailbox/item))')
            print(e)
            driver.quit()



        # Get the desired data
        try:
            # Need to Switch to iFrame first, because all the data (#document) is in under an iframe! 
            # Find the iframe element by id
            iframe_locator = WebDriverWait(driver, 60).until(
                EC.presence_of_element_located((By.ID, 'contentFrame'))
            )
            
            # Switch to the iframe context
            WebDriverWait(driver, 60).until(
                # If the frame is available it switches the given driver to the specified frame.
                EC.frame_to_be_available_and_switch_to_it(iframe_locator)
            )
            # driver now represents iframe
            ship_notice_body = WebDriverWait(driver, 60).until(
                EC.presence_of_element_located((By.XPATH, '/html/body'))
            )
            tables = WebDriverWait(ship_notice_body, 60).until(
                EC.presence_of_all_elements_located((By.TAG_NAME, 'table'))
            )

            for j, table in enumerate(tables):
                if j==0:
                    ship_notice_title = WebDriverWait(table, 60).until(
                        EC.presence_of_element_located((By.XPATH, './tbody/tr[1]/td/h1'))
                    ).text
                    print(f'Title: {ship_notice_title}')

            # # start retrieving desired data in current row/ship_notice
            # json_data_sharedAttr = dict()
            # json_data_list = []
            # for j, table in enumerate(tables):
            #     if j==0:
            #         ship_notice_title = WebDriverWait(ship_notice_body, 60).until(
            #             EC.presence_of_element_located((By.XPATH, '/tbody/tr[1]/td/h1'))
            #         ).text
            #         print(f'Title: {ship_notice_title}')

            #     elif j==1: # table1, get "Ship To"
            #         caption_data_pairs = WebDriverWait(table, 60).until(
            #             EC.presence_of_all_elements_located((By.TAG_NAME, 'tr'))
            #         )
            #         for caption_data_pair in caption_data_pairs: 
            #             caption = WebDriverWait(caption_data_pair, 60).until(
            #                             EC.presence_of_element_located((By.CLASS_NAME, 'caption'))
            #                         ).text
            #             if caption=='Ship To':
            #                 data = WebDriverWait(caption_data_pair, 60).until(
            #                                 EC.presence_of_element_located((By.CLASS_NAME, 'data'))
            #                             ).text
            #                 json_data_sharedAttr['ship_to'] = data
            #                 break
            #         assert 'ship_to' in json_data_sharedAttr.keys(), "'ship_to' key not found in json_data_sharedAttr"
            #     elif j==2: # table2, get "Ship Notice #", "Create Date/Time"
            #         caption_data_pairs = WebDriverWait(table, 60).until(
            #             EC.presence_of_all_elements_located((By.TAG_NAME, 'tr'))
            #         )
            #         for caption_data_pair in caption_data_pairs: 
            #             target_attr_count=0
            #             caption = WebDriverWait(caption_data_pair, 60).until(
            #                             EC.presence_of_element_located((By.CLASS_NAME, 'caption'))
            #                         ).text
            #             if caption=='Ship Notice #':
            #                 data = WebDriverWait(caption_data_pair, 60).until(
            #                                 EC.presence_of_element_located((By.CLASS_NAME, 'data'))
            #                             ).text
            #                 json_data_sharedAttr['ship_notice_num'] = data
            #                 target_attr_count+=1
            #             elif caption=='Create Date/Time': 
            #                 data = WebDriverWait(caption_data_pair, 60).until(
            #                                 EC.presence_of_element_located((By.CLASS_NAME, 'data'))
            #                             ).text
            #                 json_data_sharedAttr['create_datetime'] = data
            #                 target_attr_count+=1
            #             elif target_attr_count==2:
            #                 break
            #         assert 'ship_notice_num' in json_data_sharedAttr.keys(), "'ship_notice_num' key not found in json_data_sharedAttr"
            #         assert 'create_datetime' in json_data_sharedAttr.keys(), "'create_datetime' key not found in json_data_sharedAttr"
                
            #     elif j==3: # skip. (container type, master label #, returnable container #)
            #         continue
            #     else: # table4~N, items
            #         json_data_i = {}
            #         # the previous captions/keys are actually the same for all items. 
            #         caption_data_pairs = WebDriverWait(table, 60).until(
            #             EC.presence_of_all_elements_located((By.TAG_NAME, 'tr'))
            #         )
            #         for caption_data_pair in caption_data_pairs: 
            #             target_attr_count=0
            #             caption = WebDriverWait(caption_data_pair, 60).until(
            #                             EC.presence_of_element_located((By.CLASS_NAME, 'caption'))
            #                         ).text
            #             if 'Order #' in caption or 'PO #' in caption:
            #                 data = WebDriverWait(caption_data_pair, 60).until(
            #                                 EC.presence_of_element_located((By.CLASS_NAME, 'data'))
            #                             ).text
            #                 if 'Order #' in caption:
            #                     json_data_i['order_type'] = 'Buyer'
            #                 else:
            #                     json_data_i['order_type'] = 'Purchase'
            #                 json_data_i['order_num'] = data
            #                 target_attr_count+=1
            #             elif 'Buyer Part #' in caption: 
            #                 data = WebDriverWait(caption_data_pair, 60).until(
            #                                 EC.presence_of_element_located((By.CLASS_NAME, 'data'))
            #                             ).text
            #                 json_data_i['buyer_part_num'] = data
            #                 target_attr_count+=1
            #             elif 'Ship Quantity' in caption: 
            #                 data = WebDriverWait(caption_data_pair, 60).until(
            #                                 EC.presence_of_element_located((By.CLASS_NAME, 'data'))
            #                             ).text
            #                 json_data_i['ship_quantity'] = data
            #                 target_attr_count+=1
            #             elif target_attr_count==3:
            #                 break
            #         assert 'order_num' in json_data_i.keys(), "'order_num' key not found in json_data"
            #         assert 'order_type' in json_data_i.keys(), "'order_type' key not found in json_data"
            #         assert 'buyer_part_num' in json_data_i.keys(), "'buyer_part_num' key not found in json_data"
            #         assert 'ship_quantity' in json_data_i.keys(), "'ship_quantity' key not found in json_data"

            # # append a row data for a single ship notice  
            # json_data_list.append({**json_data_sharedAttr, **json_data_i})
                    

            # # single data row looks like {
            # #     "ship_to": "Location A",
            # #     "ship_notice_num": "12345",
            # #     "create_datetime": "2024-07-10 05:32:00",
            # #     "order_num": "54321",
            # #     "order_type": "Purchase or Buyer"
            # #     "buyer_part_num": "67890",
            # #     "ship_quantity": 100
            # # }
        except Exception as e:
            print('error at: Get the desired data')
            print(e)
            driver.quit()
            break

        # switch back to main mailbox page
        try:    
            # if needed to switchback to original parent frame from iframe
            # driver.switch_to.default_content()

            # Navigate back to the previous page
            driver.get(sentmail_url)

            # Wait until the URL changes back to the previous one (optional)
            WebDriverWait(driver, 60).until(EC.url_to_be(sentmail_url))
            assert WebDriverWait(driver, 60).until(EC.url_contains('mailbox/sent'))
            print('Redirected to Sent mail page!')
            assert WebDriverWait(driver, 60).until(
                EC.text_to_be_present_in_element((By.XPATH, "/html/body/div[2]/aside[2]/ol/li[2]"), "Sent")
            )
            print('Mailbox/Sent is in title! ')

        except Exception as e:
            print('error at: back to mailbox page')
            print(e)
            driver.quit()

    driver.quit()



In [112]:
# Driver Code
# Enter below your login credentials
username = "***REMOVED***"
password = "***REMOVED***"

# URL of the login page of site
# which you want to automate login.
url = "https://www.iexchangeweb.com/ieweb/general/login"

# Get the "new data": last datetime of crawled data ~ newest
date_format = "%m/%d/%Y %I:%M %p"
last_crawled_datetime = datetime.strptime("7/1/2024 2:28 PM", date_format)

# Call the function
# startScrapeBot_byRawX12(username, password, url, last_crawled_datetime, brieftest=True)
startScrapeBot_byHTMLclass(username, password, url, last_crawled_datetime, brieftest=True)

last_crawled_datetime = datetime.today()
print(f'last_crawled_datetime={last_crawled_datetime}')
# goto http://localhost:7900/?autoconnect=1&resize=scale&password=secret

Start scrape bot byHTMLclass....
It is a Brief Test! 


KeyboardInterrupt: 